In [262]:
import os
import copy
import json
import time
#import pandas as pd
#import collections
import networkx as nx
import matplotlib.pyplot as plt
#from itertools import combinations, permutations
from itertools import product

%matplotlib inline  

In [263]:
class BaseGraph:
    def __init__(self, inputs):
        self.inputs = inputs
        self.shaft_name = None
        self.edge_list = None
        self.start_list = None
        self.end_list = None
        self.graph = nx.Graph() #nx.DiGraph()
    
    def create_edge_list(self):
        # create list of edges
        elist = []
        for i, val in enumerate(self.inputs['Edges']):
            e1 = val['Node1']
            e2 = val['Node2']
            elist.append((e1, e2, ))
            
        self.edge_list = elist
        
    def create_node_list(self):
        # Create list of starting and ending nodes
        start_list = []
        end_list = []
        for i, val in enumerate(self.inputs['Nodes']):
            if val['NodeType'] == 'Shaft':
                start_list.append(val['Id'])
                self.shaft_name = val['Name']
            if val['NodeType'] == 'Vav':
                end_list.append(val['Id'])
                
        self.start_list = start_list
        self.end_list = end_list
        
    def add_edges_to_graph(self):
        self.graph.add_edges_from(self.edge_list)
        
    def set_node_demand(self, number, attribute, demand):
        self.graph.node[number][attribute] = demand
        
    def set_edge_attribute(self, n1, n2, attribute, value):
        #self.graph.edge[number][attribute] = value
        #nx.set_edge_attributes(self.graph, attribute, value)
        self.graph.edge[n1][n2][attribute] = value
        
    def set_lengths(self):
        # Create list of nodes and airflow
        edges = [x for x in self.inputs['Edges']]
        for edge in edges:
            self.set_edge_attribute(edge['Node1'], edge['Node2'], 'length', edge['Distance'])
        
    def set_airflows(self):
        # Add airflow to node data
        vav_nodes = [x for x in self.inputs['Nodes'] if x['NodeType'] == 'Vav']
        for x in vav_nodes:
            for y in self.inputs['Spaces']:
                if y['UniqueId'] == x['SpaceId']:
                    self.set_node_demand(x['Id'], 'demand', y['Airflow'])
        
    def print_graph(self):
        nx.draw(self.graph, with_labels=True, font_weight='bold')
        

In [264]:
class RouteOptions:
    def __init__(self, base_graph, shaft_name, source, targets):
        self.base_graph = base_graph
        self.shaft_name = shaft_name
        self.source = source
        self.targets = targets
        self.individual_paths = None
        self.path_combinations = None
        self.graphs = None
        self.pressure_drops = []
        
    def create_all_paths_to_each_target(self):
        paths = {}
        for target in self.targets:
            target_key = str(target)
            paths.update({target_key: []})
            for path in nx.all_simple_paths(self.base_graph, source=self.source, target=target):
                paths[target_key].append({"source": self.source, "target": target, "path": path})
                
        self.individual_paths = paths
        
    def create_all_path_combinations(self):
        # Create list of combinations of paths
        path_combinations = []
        for target in self.individual_paths:
            path_combinations.append(list(range(0, len(self.individual_paths[target]))))

        self.path_combinations = list(product(*path_combinations))
    
    def create_graph_options(self):
        # Create new graphs for each combination
        final_graphs = []

        # Loop for each path combination
        for path_combination in self.path_combinations:
            new_graph =nx.Graph() # nx.DiGraph()

            # Loop for each target
            for target_number, target in enumerate(self.targets):
                edge_list = []

                my_path = self.individual_paths[str(target)][path_combination[target_number]]['path']

                # Loop for each node in path
                for i in range(0, len(my_path)-1):
                    edge_list.append((my_path[i], my_path[i+1]))

                G1 = nx.Graph() #nx.DiGraph()
                G1.add_edges_from(edge_list)

                if target_number > 0:
                    new_graph = nx.compose(G1, new_graph)
                else:
                    new_graph = G1

            final_graphs.append(new_graph)

        self.graphs = final_graphs
        
        print('number of graphs: ' + str(len(self.graphs)))

    def set_edge_attribute(self, graph_number, n1, n2, attribute, value):
        all_edges = self.graphs[graph_number].edges()
        if (n1, n2) in all_edges or (n2, n1) in all_edges:
            self.graphs[graph_number].edge[n1][n2][attribute] = value
        
    def set_lengths(self, inputs):
        # Set lenths to edge attributes
        for i, graph in enumerate(self.graphs):
            for edge in inputs:
                self.set_edge_attribute(i, edge[0], edge[1], 'length', edge[2]['length'])
        
    def set_node_demand(self, graph_number, node_number, attribute, demand):
        self.graphs[graph_number].node[node_number][attribute] = demand
        
    def set_terminal_airflows(self, inputs):
        # Set airflow to node attributes
        for i, graph in enumerate(self.graphs):
            for node in inputs:
                if node[1]:
                    self.set_node_demand(i, node[0], 'demand', node[1]['demand'])
                    
    def calculate_airflows(self):
        for i, graph in enumerate(self.graphs):
            for target in self.targets:
                path = nx.shortest_path(graph, source=self.source, target=target)
                path.remove(target)
                
                for node in path:
                    if graph.node[target]:
                        terminal_demand = graph.node[target]['demand']
                    else:
                        terminal_demand = 0
                        
                    if graph.node[node]:
                        existing_demand = graph.node[node]['demand']
                    else:
                        existing_demand = 0
                        
                    new_demand = existing_demand + terminal_demand
                    self.set_node_demand(i, node, 'demand', new_demand)
    
    def calculate_static_pressure_drop(self):
        ## TODO THIS needs to be calculated for each duct runs to find the maximum value

        for i, graph in enumerate(self.graphs):
            total_pressure_drop = 0.0
            for target in self.targets:
                path = nx.shortest_path(graph, source=self.source, target=target)
                path_pressure_drop = 0.0
                
                # Loop for each node in path
                for i in range(0, len(path)-1):
                    # Check if nodes are in edge
                    if (path[i], path[i+1]) in graph.edges():
                        n1 = path[i]
                        n2 = path[i+1]
                        #print('Yes 1')
                    elif (path[i+1], path[i]) in graph.edges():
                        #print('Yes 2')
                        n1 = path[i+1]
                        n2 = path[i]
                    
                    #print('')
                    #print(n1)
                    #print(n2)
                    #print(graph.edge[n1][n2])
                    #print('')
                    
                    edge = graph.edges((n1, n2))
                    length = graph.edge[n1][n2]['length'] # get the length [ft]
                    friction_loss = 0.2 
                    pressure_drop = friction_loss * length/100.0  # calculate the pressure drop [in wg]
                    self.set_edge_attribute(i, n1, n2, 'pressure_drop', pressure_drop)
                    
                    print('')
                    print(i)
                    print(length)
                    print(pressure_drop)
                    print('')
                    
                    #self.graphs[i].edge[n1][n2]['pressure_drop'] = pressure_drop
                    path_pressure_drop += pressure_drop
                    
                #Compare path to total pressure drop
                total_pressure_drop = max(total_pressure_drop, path_pressure_drop)
            
            # Set the system pressure drop
            self.pressure_drops.append(total_pressure_drop)
                     
    def convert_to_direct(self):
        for i, graph in enumerate(self.graphs):
            self.graphs[i].to_directed()
    
    def print_graph_options(self):
        fig = plt.figure(figsize=(10, 5*(len(self.graphs))))
        for i, F in enumerate(self.graphs):
            ax = fig.add_subplot(len(self.graphs),1,i+1)
            nx.draw(F, with_labels=True, font_weight='bold')

        plt.show()

In [265]:
t_start = time.time()

In [266]:
#filename = 'Arch Model 3.json'
filename = 'Arch Model4.json'

In [267]:
with open(os.path.join('..', 'models', filename)) as json_data:
    inputs = json.load(json_data)

In [268]:
# Run calclations to create base mesh
base_graph = BaseGraph(inputs)
base_graph.create_edge_list()
base_graph.create_node_list()
base_graph.add_edges_to_graph()
base_graph.set_airflows()
base_graph.set_lengths()
#base_graph.print_graph()

#base_graph.graph.nodes(data=True)
#base_graph.graph.edges(data=True)

In [269]:
# Run calculations to create options
route_option_list = []
# Loop for each shaft option

for start_node in base_graph.start_list:
    print(start_node)
    route_options = RouteOptions(base_graph.graph, base_graph.shaft_name, start_node, base_graph.end_list)
    route_options.create_all_paths_to_each_target()
    route_options.create_all_path_combinations()
    route_options.create_graph_options()
    route_options.convert_to_direct()
    route_options.set_lengths(base_graph.graph.edges(data=True))
    route_options.set_terminal_airflows(base_graph.graph.nodes(data=True))
    route_options.calculate_airflows()
    route_options.calculate_static_pressure_drop()
    #route_options.print_graph_options()
    
    route_option_list.append({'start_node': start_node, 'graphs': route_options})

11
number of graphs: 32

0
8.038382725146546
0.016076765450293094


1
11.224809036897629
0.02244961807379526


2
3.628440895225383
0.007256881790450766


3
34.909941829921095
0.0698198836598422


4
12.461617274853523
0.02492323454970705


5
3.564261348553284
0.007128522697106569


6
36.4357386514468
0.0728714773028936


7
3.538225771441443
0.007076451542882886


8
6.275269979619582
0.012550539959239165


0
8.038382725146546
0.016076765450293094


1
11.224809036897629
0.02244961807379526


2
3.628440895225383
0.007256881790450766


3
34.909941829921095
0.0698198836598422


4
12.461617274853523
0.02492323454970705


5
3.564261348553284
0.007128522697106569


6
6.422782859625052
0.012845565719250105


0
8.038382725146546
0.016076765450293094


1
11.224809036897629
0.02244961807379526


2
3.628440895225383
0.007256881790450766


3
5.243639475791667
0.010487278951583334


0
8.038382725146546
0.016076765450293094


1
11.224809036897629
0.02244961807379526


2
3.628440895225383
0.007256881790

In [270]:
#base_graph.graph.edges(data=True)

In [271]:
#route_option_list[0]['graphs'].graphs[31].edges(data=True)

In [272]:
#route_option_list[0]['graphs'].calculate_static_pressure_drop()
route_option_list[0]['graphs'].graphs[31].edges(data=True)

[(11, 12, {'length': 8.038382725146546}),
 (12, 8, {'length': 25.839452311655513}),
 (8, 15, {'length': 2.935738651446856}),
 (8, 7, {'length': 6.9992344025078275}),
 (15, 2, {'length': 47.46177422855855}),
 (2, 16, {'length': 3.538225771441443}),
 (2, 1, {'length': 6.275269979619582}),
 (16, 4, {'length': 36.4357386514468}),
 (4, 13, {'length': 3.564261348553284}),
 (4, 3, {'length': 6.422782859625052}),
 (13, 10, {'length': 12.461617274853523}),
 (10, 9, {'length': 9.123317667964084}),
 (10, 6, {'length': 34.909941829921095}),
 (6, 5, {'length': 5.243639475791667})]

In [273]:
#[x for x in base_graph.graph.edges(data=True)]

In [274]:
#solution = final_graphs = route_option_list[0]['graphs'].graphs[0]
#nx.get_edge_attributes(solution,'demand')
#nx.get_node_attributes(solution,'demand')

In [276]:
data = {}
solutions = []

#SHEET_METAL = 5.0
#SHAFT = "Shaft-969767"
#COST = 15.0
STATIC_PRESSURE = 2.0
#DISTANCE = 20.0
#DIAMETER = 1.5
#AIRFLOW = 1000

friction_loss = 0.2 # in wg / 100 ft
velocity = 2000 # fpm
density = 1.1 # lb/ft2 of sheet metal
sheet_metal_unit_cost = 6.0 # USD/lb

data["Nodes"] = inputs["Nodes"]
n = 1


# TODO LOOP FOR SHAFT LOCATIIONS
final_graphs = route_option_list[0]['graphs'].graphs
shaft =  route_option_list[0]['graphs'].shaft_name

for i, solution in enumerate(final_graphs):
    edges = []
    total_weight = 0.0
    total_cost = 0.0
    
    pressure_drop = route_option_list[0]['graphs'].pressure_drops[i]
    
    for n1,n2 in solution.edges():
        if solution.node[n2]:
            airflow = solution.node[n2]['demand']
        else:
            airflow = 0
            
        cross_sectional_area = airflow/velocity  # ft2
        diameter = max(round(2.0*(cross_sectional_area/3.14)**0.5*12.0, 1), 5.0)/12.0
        cross_sectional_area = 3.14*(diameter/2)**2.0 # recalculate area
        
        #pressure_drop = friction_loss * length/100.0
        
        if solution.edge[n1][n2]:
            length = solution.edge[n1][n2]['length'];
        else:
            length = 0
            
        weight = cross_sectional_area*length*density # Sheet metal weight in lb
        total_weight += weight
        
        cost = sheet_metal_unit_cost*weight # Sheet metal cost
        total_cost += cost      
        
        edges.append({"Node1":n1, "Node2": n2, "Distance": length, "Diameter": diameter, "Airflow": airflow})
    #print(edges)
    
    solutions.append({"Id":i, "Shaft": shaft, "SheetMetal": total_weight, "Cost": total_cost, "Static Pressure": pressure_drop, "Edges":edges})

    data["Solutions"] = solutions

with open(os.path.join('..', 'models', 'solutions_' + filename), 'w') as outfile:  
    json.dump(data, outfile)

In [ ]:
"""
# Find and remove closed loop solutions

# Determine closed loops
G = final_graphs[0]
DG = nx.DiGraph(G)
simple_cycles = list(nx.simple_cycles(DG))
simple_cycles = [x for x in simple_cycles if len(x) > 2]

for closed_loop in simple_cycles:
    # Loop for each node in path
    closed_loop_list = []
    closed_loop = simple_cycles[0]
    for i in range(0, len(closed_loop)-1):
        closed_loop_list.append((closed_loop[i], closed_loop[i+1]))
    closed_loop_list.insert(0, (0, 1))    

    # Remove graph if it matches cycle
    for i, final_graph in enumerate(final_graphs):
        #print(my_closed_loop)
        if final_graphs[i].edges() == closed_loop_list:
            final_graphs.remove(final_graph)
"""